# Inference Testing for Baseline Model



## Date: 25th July 2024
## Creator: Sean Lim

The following notebook displays the baseline model used for distance estimation

The code consists of the following pipeline:
1) Yolov10 model: Used for detecting objects and is the core of the model
2) Config files: Extracts important manually input information such as real dimsnesions and the intrinsic camera matrix
- Real dimensions: Estimates in meters (m) used to compare the bounding box vs the real length, height of the object
- Intrinsic camera matrix: setting "use_own" = True allows manually input variables, while False uses the image dimensions as the camera matrix variables (not recommended)
3) The closest distance between two bounding boxes is obtained in pixel length and transforms them to 3D coordinates using computer vision transformations, obtaining the distance between the object and the camera and used as the depth to then get the location of the object (X, Y, Z=depth)
4) Euclidean distance between the center points of the boxes are obtained and if it is less then the configurable [safe_distance], a warning will be sent out and lines will be drawn

### Install Pre-requisites

In [ ]:
# !pip install -r requirements.txt

In [ ]:
# Load a pre-trained YOLOv10n model
from ultralytics import YOLOv10
from lib.Dataset import ConstructionDataset
from lib.Camera import Camera
from lib.utils import parse_yaml, estimate_distance_2d, draw_lines, estimate_distance_centers_3d
from lu_vp_detect import VPDetection
import torch
import os
from torch.utils import data
import matplotlib.pyplot as plt
import numpy as np
import cv2
# !python --version
from lu_vp_detect import VPDetection

In [ ]:
config=parse_yaml('config.yaml')

### Prepare dataset from original repo 

#### skips if config > YoloPrep > init = True

In [ ]:
# !python YoloCreate.py

In [ ]:

#Initialization
weightpath = os.path.join('runs/detect/train5/weights/best.pt')
model = YOLOv10(weightpath)
lookup = config['Classes']
config_cam = config['Camera']

batch_size = 1 # Only for testing purposes
safe_distancing = config['General']['safe_dist']
consDataset = ConstructionDataset(config, crops=True)


params = {'batch_size': batch_size,
            'shuffle':True,
            'num_workers': 6}
seed = config['General']['seed']
if seed is not None:
    torch.manual_seed(config['General']['seed'])
generator = data.DataLoader(consDataset, **params)

# Drawing parameters
Color_palette = 255 * np.eye(3)
config_cam = config['Camera']
config_vp = config['VP_Detector']
remove_fisheye=config_cam['remove_fisheye']
# BGR format
# First row is red, second green, third blue
Color_palette = Color_palette[:, ::-1].astype(int).tolist()

length_thresh = config_vp['length_thresh']
principal_point = config_vp['principal_point']
focal_length = config_vp['focal_length']
seed = config['General']['seed'] # Or specify whatever ID you want (integer)

vpd = VPDetection(length_thresh, seed=seed)


In [ ]:
# Initialise generator
generator_iter = iter(generator)

## Test Loop

In [ ]:
_, _, _, local_image, local_image_tensor = next(generator_iter)
temp_img = local_image_tensor[0]
temp_img_ori = np.array(local_image[0])

print(temp_img_ori.shape)

cam = Camera(use_own = config_cam['use_own'], img=temp_img_ori, distortion_coef=config_cam['distortion_coef'], fx=config_cam['fx'], fy=config_cam['fy'], cx=config_cam['cx'], cy=config_cam['cy'])


with torch.no_grad():
    results = model(temp_img)

#Predicted coordinates of box (top left, bottom right)
list_boxes = results[0].boxes.xyxy

#List of detected classes
detected_classes = results[0].boxes.cls


# Plot original Image with Yolo Detection    
img = results[0].plot()

#Draw lines and send warning if a distance is lower than safe distance
if(len(list_boxes)>1 and 0 in detected_classes):
    list_conf = results[0].boxes.conf
    list_workers = [(i, detected_classes[i].item(), list_boxes[i], list_conf[i].item()) for i in range(len(detected_classes)) if int(detected_classes[i]) == 0]
    list_nonworkers = [(i, detected_classes[i].item(), list_boxes[i], list_conf[i].item()) for i in range(len(detected_classes)) if int(detected_classes[i]) != 0]

##################################################################################################################################################
# List workers and List Non Workers have the following:
# (0: index wrt detected objects in YOLO accordingly, 1: Class in int format, 2: xyxy, 3: Confidence Score for ease of labelling when plotting)
##################################################################################################################################################

vps = vpd.find_vps(temp_img_ori)
print(vps)
print(vpd.vps_2D)

vp1, vp2, vp3 = vpd.vps_2D[:3]

# draw_grid(img, vp1, vp2, vp3, Color_palette)

for worker in list_workers:
    for nonworker in list_nonworkers:
        length= estimate_distance_2d(worker[2], nonworker[2])
        # print(length)
        hazard = lookup[nonworker[1]]['name']
        worker_dim = lookup[worker[1]]['dimensions']
        hazard_dim = lookup[nonworker[1]]['dimensions']
        
        worker_3d_coords = cam.find_real_coords(worker[2], worker_dim)
        hazard_3d_coords = cam.find_real_coords(nonworker[2], hazard_dim)
        distance = estimate_distance_centers_3d(worker_3d_coords, hazard_3d_coords)
        
        
        # print(f'3D Coordinates of worker: ,{worker[3]:.2f},{worker_3d_coords}')
        # print(f'3D Coordinates of , {hazard}: , {nonworker[3]:.2f}, {hazard_3d_coords}')
        # print(f'Distance = {distance:.2f}m')
        
        if(distance < safe_distancing):
            print(f'Unsafe distancing between worker:{worker[3]:.2f} and {hazard}:{nonworker[3]:.2f}, Distance={distance:.2f}m')
            print('3D Coordinates of worker: ',worker[3],worker_3d_coords)
            print('3D Coordinates of ', hazard,': ', nonworker[3], hazard_3d_coords)
            color = (255,0,0)
        else:
            color = (255,255,255)
        
        
        draw_lines(worker[2], nonworker[2], img, distance, color)
        
print('__________________________________________________________________________')


plt.figure(figsize=(18, 16), dpi=80)
plt.imshow(img)
